# Setup

In [ ]:
!nvidia-smi

Mon Dec 22 15:08:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install -Uq transformers datasets accelerate evaluate scikit-learn huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 20.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# In order to access gated model
from huggingface_hub import login
login(new_session=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
import torch

train_df = pd.read_csv("/content/drive/MyDrive/suicide_detection/bert_train.csv")
val_df = pd.read_csv("/content/drive/MyDrive/suicide_detection/bert_val.csv")
test_df = pd.read_csv("/content/drive/MyDrive/suicide_detection/bert_test.csv")

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

dataset = DatasetDict({
    "train": train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})



print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 167021
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 18558
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 46395
    })
})


In [ ]:
from transformers import AutoTokenizer

model_name = "mental/mental-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
  return tokenizer(
      examples['text'],
      padding='max_length',
      truncation=True,
      max_length=128
  )

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenized_datasets = tokenized_datasets.remove_columns(['text'])
tokenized_datasets = tokenized_datasets.rename_column('label', 'labels')

tokenized_datasets.set_format('torch')
print(tokenized_datasets['train'][0])


tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/167021 [00:00<?, ? examples/s]

Map:   0%|          | 0/18558 [00:00<?, ? examples/s]

Map:   0%|          | 0/46395 [00:00<?, ? examples/s]

{'labels': tensor(1), 'input_ids': tensor([  101,  2054,  6433,  2043,  2017,  1005,  2128,  3236,  1029,  1045,
         1005,  2310,  2196,  2042,  3236,  2076,  1013,  2044,  2667,  1012,
         1045,  2113,  2017,  2064,  2203,  2039,  2006,  1037,  8827, 17994,
         2907,  1012,  2515,  2619,  2175,  2083,  2115,  3042,  1998,  4933,
         2065,  2017,  2203,  2039,  4129,  2068,  2111,  2024,  1996,  3114,
         2017,  2699,  1029,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 

In [ ]:
from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, recall_score

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)

  precision, recall, f1, _ = precision_recall_fscore_support(
      labels, preds, average='binary', pos_label=1
  )
  acc = accuracy_score(labels, preds)
  suicide_recall = recall_score(labels, preds, pos_label=1)

  return {
      'accuracy': acc,
      'precision': precision,
      'recall': recall,
      'f1': f1,
      'suicide_recall': suicide_recall
  }

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/suicide_bert_model',
    eval_strategy='steps',
    eval_steps=500,
    save_strategy='steps',
    save_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='suicide_recall',
    greater_is_better=True,
    logging_dir='/content/drive/MyDrive/logs',
    logging_steps=100,
    warmup_steps=500,
    fp16=True,
    save_total_limit=2,
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics
)

config.json:   0%|          | 0.00/639 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mental/mental-bert-base-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print("training...")
trainer.train()

trainer.save_model("/content/drive/MyDrive/suicide_bert_final")
tokenizer.save_pretrained('/content/drive/MyDrive/suicide_bert_final')
print("complted training. model saved to drive!")

training...


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Suicide Recall
500,0.104800,0.105374,0.964490,0.975516,0.952920,0.964085,0.952920
1000,0.104800,0.085165,0.971387,0.977936,0.964555,0.971199,0.964555
1500,0.076800,0.095087,0.971333,0.962083,0.981362,0.971627,0.981362
2000,0.094700,0.077261,0.974890,0.975205,0.974574,0.974890,0.974574
2500,0.064100,0.080738,0.975752,0.975657,0.975867,0.975762,0.975867
3000,0.072500,0.084056,0.974782,0.983648,0.965632,0.974557,0.965632
3500,0.084900,0.068485,0.976452,0.974976,0.978022,0.976497,0.978022
4000,0.062300,0.074429,0.975698,0.967100,0.984917,0.975927,0.984917
4500,0.071300,0.072016,0.976991,0.985205,0.968541,0.976802,0.968541
5000,0.078900,0.067866,0.977476,0.983104,0.971666,0.977352,0.971666


complted training. model saved to drive!


In [ ]:
test_results = trainer.evaluate(tokenized_datasets['test'])
print("\ntest set results:")
for key,value in test_results.items():
  print(f"{key}: {value:.4f}")

predictions = trainer.predict(tokenized_datasets['test'])
y_pred = predictions.predictions.argmax(-1)
y_pred_proba = torch.softmax(torch.tensor(predictions.predictions), dim=1)[:, 1].numpy()

np.save('/content/drive/MyDrive/bert_test_proba.npy', y_pred_proba)
np.save('/content/drive/MyDrive/bert_test_pred.npy', y_pred)

print('saved the predictions.')



test set results:
eval_loss: 0.0681
eval_accuracy: 0.9765
eval_precision: 0.9684
eval_recall: 0.9853
eval_f1: 0.9768
eval_suicide_recall: 0.9853
eval_runtime: 77.9202
eval_samples_per_second: 595.4170
eval_steps_per_second: 9.3040
epoch: 3.0000
saved the predictions.
